In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.metrics import *

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [8]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 175ms/step - loss: 0.7334 - accuracy: 0.5465 - precision: 0.5185 - recall: 0.4579 - auc: 0.5792
Epoch 2/30
58/58 [==============================] - 10s 173ms/step - loss: 0.6102 - accuracy: 0.6637 - precision: 0.6494 - recall: 0.6145 - auc: 0.7273
Epoch 3/30
58/58 [==============================] - 10s 176ms/step - loss: 0.5285 - accuracy: 0.7130 - precision: 0.6904 - recall: 0.7033 - auc: 0.8075
Epoch 4/30
58/58 [==============================] - 10s 175ms/step - loss: 0.4478 - accuracy: 0.7919 - precision: 0.7793 - recall: 0.7757 - auc: 0.8722
Epoch 5/30
58/58 [==============================] - 10s 175ms/step - loss: 0.3786 - accuracy: 0.8335 - precision: 0.8136 - recall: 0.8364 - auc: 0.9145
Epoch 6/30
58/58 [==============================] - 10s 177ms/step - loss: 0.3270 - accuracy: 0.8576 - precision: 0.8565 - recall: 0.8364 - auc: 0.9360
Epoch 7/30
58/58 [==============================] - 10s 177ms/step - loss: 0.2770 - accu

58/58 [==============================] - 10s 178ms/step - loss: 0.0237 - accuracy: 0.9934 - precision_1: 0.9930 - recall_1: 0.9930 - auc_1: 0.9997
Epoch 24/30
58/58 [==============================] - 10s 178ms/step - loss: 0.0352 - accuracy: 0.9880 - precision_1: 0.9815 - recall_1: 0.9930 - auc_1: 0.9991
Epoch 25/30
58/58 [==============================] - 10s 180ms/step - loss: 0.0976 - accuracy: 0.9650 - precision_1: 0.9648 - recall_1: 0.9603 - auc_1: 0.9934
Epoch 26/30
58/58 [==============================] - 10s 179ms/step - loss: 0.0671 - accuracy: 0.9726 - precision_1: 0.9654 - recall_1: 0.9766 - auc_1: 0.9971
Epoch 27/30
58/58 [==============================] - 10s 180ms/step - loss: 0.0458 - accuracy: 0.9825 - precision_1: 0.9836 - recall_1: 0.9790 - auc_1: 0.9988
Epoch 28/30
58/58 [==============================] - 10s 178ms/step - loss: 0.0338 - accuracy: 0.9890 - precision_1: 0.9883 - recall_1: 0.9883 - auc_1: 0.9993
Epoch 29/30
58/58 [==============================] - 10s 1

58/58 [==============================] - 10s 178ms/step - loss: 0.2436 - accuracy: 0.8949 - precision_3: 0.8722 - recall_3: 0.9089 - auc_3: 0.9641
Epoch 15/30
58/58 [==============================] - 10s 180ms/step - loss: 0.2070 - accuracy: 0.9146 - precision_3: 0.8977 - recall_3: 0.9229 - auc_3: 0.9744
Epoch 16/30
58/58 [==============================] - 10s 179ms/step - loss: 0.1785 - accuracy: 0.9288 - precision_3: 0.9211 - recall_3: 0.9276 - auc_3: 0.9807
Epoch 17/30
58/58 [==============================] - 10s 179ms/step - loss: 0.1413 - accuracy: 0.9419 - precision_3: 0.9433 - recall_3: 0.9322 - auc_3: 0.9887
Epoch 18/30
58/58 [==============================] - 10s 178ms/step - loss: 0.1199 - accuracy: 0.9595 - precision_3: 0.9666 - recall_3: 0.9463 - auc_3: 0.9909
Epoch 19/30
58/58 [==============================] - 10s 178ms/step - loss: 0.1107 - accuracy: 0.9606 - precision_3: 0.9558 - recall_3: 0.9603 - auc_3: 0.9928
Epoch 20/30
58/58 [==============================] - 10s 1

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 314ms/step - loss: 0.6342 - accuracy: 0.6298 - precision_5: 0.6355 - recall_5: 0.4930 - auc_5: 0.6849
Epoch 2/30
58/58 [==============================] - 18s 316ms/step - loss: 0.4488 - accuracy: 0.8007 - precision_5: 0.7834 - recall_5: 0.7944 - auc_5: 0.8714
Epoch 3/30
58/58 [==============================] - 18s 313ms/step - loss: 0.3070 - accuracy: 0.8784 - precision_5: 0.8578 - recall_5: 0.8879 - auc_5: 0.9414
Epoch 4/30
58/58 [==============================] - 18s 313ms/step - loss: 0.2167 - accuracy: 0.9211 - precision_5: 0.9009 - recall_5: 0.9346 - auc_5: 0.9701
Epoch 5/30
58/58 [==============================] - 18s 315ms/step - loss: 0.1976 - accuracy: 0.9266 - precision_5: 0.9093 - recall_5: 0.9369 - auc_5: 0.9755
Epoch 6/30
58/58 [==============================] - 18s 313ms/step - loss: 0.1262 - accuracy: 0.9595 - precision_5: 0.9622 - recall_5: 0.9509 - auc_5: 0.9893
Epoch 7/30
58/58 [==============================] - 

58/58 [==============================] - 18s 313ms/step - loss: 0.0600 - accuracy: 0.9792 - precision_6: 0.9745 - recall_6: 0.9813 - auc_6: 0.9974
Epoch 23/30
58/58 [==============================] - 18s 313ms/step - loss: 0.0437 - accuracy: 0.9847 - precision_6: 0.9859 - recall_6: 0.9813 - auc_6: 0.9987
Epoch 24/30
58/58 [==============================] - 18s 309ms/step - loss: 0.0161 - accuracy: 0.9956 - precision_6: 0.9977 - recall_6: 0.9930 - auc_6: 0.9998
Epoch 25/30
58/58 [==============================] - 18s 314ms/step - loss: 0.0071 - accuracy: 0.9978 - precision_6: 0.9977 - recall_6: 0.9977 - auc_6: 1.0000
Epoch 26/30
58/58 [==============================] - 18s 310ms/step - loss: 0.0061 - accuracy: 0.9989 - precision_6: 0.9977 - recall_6: 1.0000 - auc_6: 1.0000
Epoch 27/30
58/58 [==============================] - 18s 312ms/step - loss: 0.0069 - accuracy: 0.9978 - precision_6: 0.9977 - recall_6: 0.9977 - auc_6: 1.0000
Epoch 28/30
58/58 [==============================] - 18s 3

58/58 [==============================] - 18s 313ms/step - loss: 0.0694 - accuracy: 0.9726 - precision_8: 0.9675 - recall_8: 0.9743 - auc_8: 0.9963
Epoch 14/30
58/58 [==============================] - 18s 309ms/step - loss: 0.1176 - accuracy: 0.9671 - precision_8: 0.9693 - recall_8: 0.9603 - auc_8: 0.9894
Epoch 15/30
58/58 [==============================] - 18s 311ms/step - loss: 0.0957 - accuracy: 0.9639 - precision_8: 0.9604 - recall_8: 0.9626 - auc_8: 0.9939
Epoch 16/30
58/58 [==============================] - 18s 312ms/step - loss: 0.0523 - accuracy: 0.9814 - precision_8: 0.9813 - recall_8: 0.9790 - auc_8: 0.9971
Epoch 17/30
58/58 [==============================] - 18s 315ms/step - loss: 0.0388 - accuracy: 0.9880 - precision_8: 0.9883 - recall_8: 0.9860 - auc_8: 0.9991
Epoch 18/30
58/58 [==============================] - 18s 311ms/step - loss: 0.0380 - accuracy: 0.9814 - precision_8: 0.9724 - recall_8: 0.9883 - auc_8: 0.9992
Epoch 19/30
58/58 [==============================] - 18s 3